In [9]:
import psycopg as pg
from sklearn.feature_extraction.text import TfidfTransformer


In [2]:
conn = pg.connect( dbname= "postgres", user="postgres", password="password", host="127.0.0.1", port="5432")

In [3]:
conn.autocommit = True

In [4]:
cursor = conn.cursor()

In [5]:
sql = '''DROP database vector'''
cursor.execute(sql)
sql = '''CREATE database vector'''
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=127.0.0.1 database=postgres) at 0x2f7ed252770>

In [6]:
conn.close()

In [11]:
dbname = None
user = None
password = None
host = None
port = None


SyntaxError: invalid syntax (4149504281.py, line 1)

In [7]:
conn = pg.connect(dbname= dbname, user= user, password= password, host= host, port= port)

In [8]:
conn.execute('CREATE EXTENSION IF NOT EXISTS vector') #la instalación en windows es una lata, instalar VSTUDIO 2022

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=127.0.0.1 user=postgres database=vector) at 0x2f7ed252820>

In [ ]:
import timeit
class DatabaseBenchmark:
    """
    Clase para benchmark de postgres
    Falta arreglar las querys para tipo python, decidir tipo de indexacion y ver como funciona con datos reales
    al contrario de chromi, no encuentro vectorizador por default, por lo que se puede utilizar tfidf
    """
    def __init__(self, db_connection, cost_per_hour = False):3
        """
        db_connection: Conexión a base de datos
        """
        self.db_connection = db_connection
        self.cost_per_hour = cost_per_hour

    def query(self, query_string, n_results):
        """
        query_string: vector de representación de un texto
        n_results: numeros de vecinos a buscar
        """
        with db_connection.cursor() as cur:
            
            query = "SELECT * FROM items ORDER BY embedding <-> '{0}' LIMIT {1};".format(query_string, n_results) 
        
            results = cur.execute(query)

            return [record for record in results]  
#       

    def build_index(self, vectorcol):
        """
        incompleta, probar con datos
        """
        query = 'CREATE INDEX ON {0} USING hnsw (embedding vector_l2_ops);'.format(vectorcol)
        
#       

    def measure_qps(self, query string, num_queries=5, n_results = 1):
        
        start_time = timeit.default_timer()
        for _ in range(num_queries):
            self.query(query_string= query string, n_results= n_results)
        end_time = timeit.default_timer()
        
        qps = num_queries / (end_time - start_time)
        
        return qps
    
    def measure_qp_dollar(self, qps):
        if self.cost_per_hour is False:
            return "No hay costo por hora"
        qp_dollar = (qps / self.cost_per_hour) * 3600
        return qp_dollar
    
    def measure_latency(self, query string, ):

        
        start_time = timeit.default_timer()
        self.query(query string, 1)
        latency = timeit.default_timer() - start_time
        
        return latency
    
    def measure_index_building_time(self):

        
        start_time = timeit.default_timer()
        self.build_index(["Esto es un documento"], {"source": "my_source"}, ["id1"])
        index_building_time = timeit.default_timer() - start_time
        
        return index_building_time